In [1]:
import re
import os 
import sys
import numpy as np 
import time
import random

In [18]:
lang  = 'cwe' # 'sw'
cwe = open(os.path.join('data',lang+'-train.txt'), 'r').read().lower()
n_gram_value = 5

In [19]:
chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'

In [20]:
character_list = re.findall(r"[%s]"%chars, cwe)

##  The reference for this implementation is: 
1. https://towardsdatascience.com/text-generation-using-n-gram-model-8d12d9802aa0
2. https://towardsdatascience.com/text-generation-using-n-gram-model-8d12d9802aa0

In [232]:

def tokenize(text: str) -> list[str]:
    """
    :param text: Takes input sentence
    :return: tokenized sentence
    """
    for punct in string.punctuation:
        text = text.replace(punct, ' '+punct+' ')
    t = text.split()
    return t

def get_ngrams(n: int, tokens: list) -> list:
    """
    :param n: n-gram size
    :param tokens: tokenized sentence
    :return: list of ngrams
    ngrams of tuple form: ((previous wordS!), target word)
    """
    tokens = (n-1)*['<START>'] + tokens
    l = [(tuple([tokens[i-p-1] for p in reversed(range(n-1))]), tokens[i]) for i in range(n-1, len(tokens))]
    return l

In [233]:
class NgramModel(object):

    def __init__(self, n):
        self.n = n

        # dictionary that keeps list of candidate words given context
        self.context = {}

        # keeps track of how many times ngram has appeared in the text before
        self.ngram_counter = {}

    def update(self, sentence: str) -> None:
        """
        Updates Language Model
        :param sentence: input text
        """
        n = self.n
        ngrams = []

        ngrams = get_ngrams(n, sentence)
            
        for ngram in ngrams:
            if ngram in self.ngram_counter:
                self.ngram_counter[ngram] += 1.0
            else:
                self.ngram_counter[ngram] = 1.0

            prev_words, target_word = ngram
            if prev_words in self.context:
                self.context[prev_words].append(target_word)
            else:
                self.context[prev_words] = [target_word]
                
    def prob(self, context, token):
        """
        Calculates probability of a candidate token to be generated given a context
        :return: conditional probability
        """
        try:
            count_of_token = self.ngram_counter[(context, token)]
            count_of_context = float(len(self.context[context]))
            result = count_of_token / count_of_context

        except KeyError:
            result = 0.0
        return result
    
    def random_token(self, context):
        """
        Given a context we "semi-randomly" select the next word to append in a sequence
        :param context:
        :return:
        """
        r = random.random()
        map_to_probs = {}
        token_of_interest = self.context[context] # this can be all the tokens 
        temp_prob_list = []
        for token in token_of_interest:
            map_to_probs[token] = self.prob(context, token)

        map_to_probs = {k: v for k, v in sorted(map_to_probs.items(), key=lambda item: item[1])}
        token = np.random.choice(list(map_to_probs.keys()))
        prob = map_to_probs[token]
        return token, prob
#         for token in map_to_probs.keys()[:3]:
#             prob = map_to_probs[token]
#             return token, prob
#         summ = 0
#         for token in sorted(map_to_probs):
#             summ += map_to_probs[token]
#             if summ > r:
#                 return token, summ
            

    def generate_text(self, token_count: int, history):
        """
        :param token_count: number of words to be produced
        :return: generated text
        """
        if type(history)==str:
            history = list(history)
        n = self.n
#         context_queue = (n - len(history) -1 ) * ['<START>'] + list(history)
        context_queue = (n -1 ) * ['<START>'] 

        result = []
        for _ in range(token_count):
            obj, prob = self.random_token(tuple(context_queue))
            result.append(obj)
            if n > 1:
                context_queue.pop(0)
                if obj == '.':
                    context_queue = (n - 1) * ['<START>']
                else:
                    context_queue.append(obj)

        return ''.join(result)

In [223]:
chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'

In [224]:
def create_ngram_model(n, text_list):
    m = NgramModel(n)
    m.update(text_list)
    return m

In [234]:
if __name__ == "__main__":
    start = time.time()
    lang  = 'cwe' # 'sw'
#     cwe = open(os.path.join('data',lang+'-train.txt'), 'r').read().lower().split('.')
# #     chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'
# #     character_list = re.findall(r"[%s]"%chars, cwe)

    
#     for i in range(len(cwe)):
#         cwe[i] = list(cwe[i].strip(' ')) + ['.'] # .strip(' ')
    cwe = open(os.path.join('data',lang+'-train.txt'), 'r').read().lower()   
    cwe = list(cwe)
        
    
    from_scratch_model = create_ngram_model(n_gram_value, cwe)

    print (f'Language Model creating time: {time.time() - start}')
    print()
    start = time.time()
    random.seed(5)
    
#     print(f'{"="*50}\nGenerated text:')
    print(from_scratch_model.generate_text(100,'s'))
#     print(f'{"="*50}')

Language Model creating time: 1.2683687210083008

chike.chike.chike.chika, 'sauli," balutangaiko, 'ichupa yambelusangaja vonisigwa.chikomwike leme, bu


In [258]:
def evaluate_one(lang):
    testfile = open(os.path.join('data',lang+'-test.txt'), 'r')
    max_history = n_gram_value
#     history = (n_gram_value-1) * ['<START>']
    history = list(testfile.read(n_gram_value-1))
    loss_anything_goes = 0
    loss_from_scratch = 0
    count = 0
    states = None
    total_loss_list = []
    while count<n_gram_value:
        c = testfile.read(1)
        if not c:
            break
        try:
            count += 1 
            t, prob = from_scratch_model.random_token(tuple(history))
            loss_from_scratch -= np.log2(prob)
        except:
            pass
        if len(history) == max_history:
            history.pop(0)
        history.append(c)
    return loss_from_scratch/count
     



In [259]:
evaluate_one(lang)

1.7988706873717717

In [260]:
n_gram_value = 5
start = time.time()
cwe = open(os.path.join('data',lang+'-train.txt'), 'r').read().lower()   
cwe = list(cwe)

from_scratch_model = create_ngram_model(n_gram_value, cwe)

print (f'Language Model creating time: {time.time() - start}')
print()
start = time.time()
random.seed(5)

print(from_scratch_model.generate_text(100,'s'))

evaluate_one(lang)


Language Model creating time: 1.3852636814117432

chikaweke lwa ndodo.chikutwa kolazigwe,' nawandewa,' kwandewa uchinyendaza lonzi.chikwalazengo hamag


1.4818781872275404

In [267]:
n_gram_value = 9
start = time.time()
cwe = open(os.path.join('data',lang+'-train.txt'), 'r').read().lower()   
cwe = list(cwe)

from_scratch_model = create_ngram_model(n_gram_value, cwe)

print (f'Language Model creating time: {time.time() - start}')
print()
start = time.time()
random.seed(5)

print(from_scratch_model.generate_text(100,'s'))

evaluate_one(lang)

Language Model creating time: 2.032823085784912

chikale vinogile.chikale chiuvwizi wowose kulawana ningo iyeng'higwa, na muhatosa, mulekeni muhe yol


0.1754618754488834

In [270]:
n_gram_value = 6
start = time.time()
cwe = open(os.path.join('data',lang+'-train.txt'), 'r').read().lower()   
cwe = list(cwe)

from_scratch_model = create_ngram_model(n_gram_value, cwe)

print (f'Language Model creating time: {time.time() - start}')
print()
start = time.time()
random.seed(5)

print(from_scratch_model.generate_text(100,'s'))

evaluate_one(lang)

Language Model creating time: 1.3722567558288574

chikangagile.chikasasala, hudaha wela weye eliya, one vichala kaye, umuwonile hishima.chikagaya, bwa


0.00048428852923516316